# Creating Grammars for speech hints

This tutorial walks you through the process of creating custom speech hint grammars for use in inverse text normalization.
The primary application of speech hint grammars is to provide specific normalization post Automatic Speech Recognition (ASR).

## Dependencies
You need to download the [speech hints grammars](https://github.com/nvidia-riva/speech-hints)

## Prerequisites
This tutorial assumes that you are familiar with finite state acceptors and transducers. Familiarity with the [Pynini](https://www.openfst.org/twiki/bin/view/GRM/Pynini) library and Nemo's [WFST Tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/text_processing/WFST_Tutorial.ipynb) is assumed.

## Overview
Functionally, speech hints should at the bare minimum have the following:
1. A passthrough finite-state transducer (FST) that transduces the input text as is ($\Sigma *$ - a set of all possible strings over the alphabet $\Sigma$). This FST should have the longest distance/weight relative to the other FSTs
2. An FST per class of interest. FST for a class can import from other FSTs. However, the resultant FST when exported will be independent of FSTs it imports from.

Grammars in speech hints are composed on the fly. Grammars can be standalone references to the FST or consist of FSTs in context with sentences. Grammars are denoted by \\$\<FSTNAME\>. For English the following [grammars](https://github.com/nvidia-riva/speech-hints) are supported
1. `$OOV_NUMERIC_SEQUENCE`
2. `$OOV_ALPHA_SEQUENCE`
3. `$OOV_ALPHA_NUMERIC_SEQUENCE`
4. `$FULLPHONENUM`
5. `$POSTALCODE`
6. `$OOV_CLASS_ORDINAL`
7. `$OOV_CLASS_NUMERIC`
8. `$PERCENT`
9. `$TIME`
10. `$MONEY`
11. `$MONTH`
12. `$DAY`

## Using Existing Speech Hint Grammars in Python


In [14]:
from speech_hint import apply_hint
import pynini
from pynini.lib import pynutil

In [10]:
# Applying `$FULLPHONENUM` grammar on the input text
apply_hint("one eight hundred five five five four oh oh one","$FULLPHONENUM")

'1-800-555-4001'

In [11]:
apply_hint("my phone number is one eight hundred five five five four oh oh one","$FULLPHONENUM")

'my phone number is 1-800-555-4001'

In [8]:
# Specifying `$FULLPHONENUM` grammar in context
apply_hint("my phone number is one eight hundred five five five four oh oh one","my phone number is $FULLPHONENUM")

'my phone number is 1-800-555-4001'

In [13]:
apply_hint("I think my phone number is one eight hundred five five five four oh oh one","my phone number is $FULLPHONENUM")

'I think my phone number is 1-800-555-4001'

In [9]:
# Specifying `$FULLPHONENUM` grammar in Context - Context does not match
apply_hint("my phone number is one eight hundred five five five four oh oh one","my phone number is not $FULLPHONENUM")

'my phone number is one eight hundred five five five four oh oh one'

## Sample Grammar for Handling Alphabet Sequences
Let's say we need to build a grammar to support the conversion of alphabet sequences to a single word ('i b m' -> 'ibm').  (For a more detailed implementation, refer to https://github.com/nvidia-riva/speech-hints/blob/main/en/oov_class_alpha_sequence.py)

In [15]:
# Function to Apply FST
def apply_fst(utterance, fst):
    try:
        return pynini.shortestpath(utterance @ fst).string().strip()
    except pynini.FstOpError:
        print(f"Error: No valid output with given input: '{utterance}, {fst}'")


In [21]:
from en.primitives import NEMO_ALPHA, NEMO_WHITE_SPACE
character = NEMO_ALPHA
word_fst = pynini.closure(character)
sequence = character + pynini.closure(pynutil.delete(" ") + character, 1)
fst = sequence @ (word_fst)

apply_fst('i b m', fst)

'ibm'

To use the custom FST (grammar) in `speech_hints`, add it with a suitable name to `fst_dict` in `speech_hints.py`. You can then export the grammars as an FST Archive (`.far`) file using the `export_to_far.py` script.